In [1]:
import ssl

import tensorflow as tf
import pandas as pd
import numpy as np

ssl._create_default_https_context = ssl._create_unverified_context

csv_file = tf.keras.utils.get_file('Coinbase_BTCUSD_1h.csv', 'https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
data_frame = pd.read_csv(csv_file, skiprows=1, index_col="Date")

data_frame.index = pd.to_datetime(data_frame.index, format='%Y-%m-%d %I-%p')

data_frame.head()

/home/chuba/miniconda3/envs/tensortrade/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chuba/miniconda3/envs/tensortrade/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chuba/miniconda3/envs/tensortrade/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chuba/miniconda3/envs/ten

,Symbol,Open,High,Low,Close,Volume BTC,Volume USD
Date,,,,,,,
2019-10-08 10:00:00,BTCUSD,8184.86,8212.09,8184.85,8212.09,38.26,313516.88
2019-10-08 09:00:00,BTCUSD,8172.71,8186.31,8151.84,8184.86,134.29,1096797.85
2019-10-08 08:00:00,BTCUSD,8218.95,8218.95,8151.94,8172.71,560.26,4582942.89
2019-10-08 07:00:00,BTCUSD,8245.17,8245.17,8210.98,8218.95,101.21,832894.13
2019-10-08 06:00:00,BTCUSD,8258.19,8263.84,8238.52,8245.17,99.48,820497.47


In [5]:
len(data_frame)

19896

In [2]:
prepared_df = data_frame.drop('Symbol', axis=1)
prepared_df['Date'] = prepared_df.index.values.astype(np.int64) // 10 ** 9
prepared_df = prepared_df.sort_index()

print(prepared_df.head())

train_dataset = tf.data.Dataset.from_tensor_slices(dict(prepared_df))

                        Open     High      Low    Close  Volume BTC  \
Date                                                                  
2017-07-01 11:00:00  2505.56  2513.38  2495.12  2509.17      114.60   
2017-07-01 12:00:00  2509.17  2512.87  2484.99  2488.43      157.36   
2017-07-01 13:00:00  2488.43  2488.43  2454.40  2454.43      280.28   
2017-07-01 14:00:00  2454.43  2473.93  2450.83  2459.35      289.42   
2017-07-01 15:00:00  2459.35  2475.00  2450.00  2467.83      276.82   

                     Volume USD        Date  
Date                                         
2017-07-01 11:00:00   287000.32  1498906800  
2017-07-01 12:00:00   393142.50  1498910400  
2017-07-01 13:00:00   693254.01  1498914000  
2017-07-01 14:00:00   712864.80  1498917600  
2017-07-01 15:00:00   682105.41  1498921200  


In [5]:
train_dataset.take(5)

<DatasetV1Adapter shapes: {Open: (), High: (), Low: (), Close: (), Volume BTC: (), Volume USD: (), Date: ()}, types: {Open: tf.float64, High: tf.float64, Low: tf.float64, Close: tf.float64, Volume BTC: tf.float64, Volume USD: tf.float64, Date: tf.int64}>

In [ ]:
ds_iter = iter(train_dataset)

while True:
    batch = next(ds_iter)
    print(batch['Close'])

Tensor("IteratorGetNext_33207:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33208:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33209:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33210:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33211:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33212:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33213:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33214:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33215:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33216:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33217:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33218:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33219:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33220:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33221:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33222:0", shape=(), dtype=float64)
Tensor("IteratorGetNext_33223:0", shape=(), dtype=float6

In [20]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(prepared_df, test_size=0.2)

volume = tf.feature_column.numeric_column("Volume USD")
volume_buckets = tf.feature_column.bucketized_column(volume, [1000, 10000, 50000, 100000, 250000, 500000, 1000000, 2500000, 5000000, 10000000])

feature_layer = tf.keras.layers.DenseFeatures(volume_buckets)

train_labels = feature_layer(dict(train_df)).numpy()
test_labels = feature_layer(dict(test_df)).numpy()

train_labels, test_labels

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [29]:
no_volume_train = train_df.drop(['Volume BTC', 'Volume USD'], axis=1)
no_volume_test = test_df.drop(['Volume BTC', 'Volume USD'], axis=1)

train_ds = tf.data.Dataset.from_tensor_slices((no_volume_train.values, train_labels)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((no_volume_test.values, test_labels)).shuffle(1024).batch(32)

model = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(11),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])

model.fit(train_ds)

loss, accuracy = model.evaluate(test_ds)

116/116 [==============================] - 0s 1ms/step - loss: 4.8328 - accuracy: 0.1437


In [ ]:
import tensorflow as tf
import numpy as np

from IPython.display import SVG, Image
from tensorflow.keras.utils import plot_model

items_per_gen = 1000
n_samples = 100
output_shape=(items_per_gen, 1)

generator = tf.keras.Sequential([
            tf.keras.layers.Dense(units=256, input_shape=(1, n_samples), activation="relu"),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.BatchNormalization(momentum=0.8),
            tf.keras.layers.Dense(512),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.BatchNormalization(momentum=0.8),
            tf.keras.layers.Dense(1024),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.BatchNormalization(momentum=0.8),
            tf.keras.layers.Dense(np.prod(output_shape), activation='tanh'),
            tf.keras.layers.Reshape(output_shape)
        ])

discriminator = tf.keras.Sequential([
    tf.keras.layers.LSTM(512, input_shape=output_shape, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512)),
    tf.keras.layers.Dense(512),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dense(256),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [ ]:
plot_model(generator, to_file='test_keras_plot_model.png', show_shapes=True)
Image('test_keras_plot_model.png')

In [ ]:
plot_model(discriminator, to_file='test_keras_plot_model_1.png', show_shapes=True)
Image('test_keras_plot_model_1.png')